### 라이브러리 정리

In [ ]:
# # 1. 필수 라이브러리 설치
# !pip install -q transformers torch accelerate sentencepiece
# !pip install -q yt-dlp pydub SpeechRecognition webrtcvad deepface
# !pip install -q opencv-python ultralytics

### LLM(LG EXAONE) 켜기

In [ ]:
# from huggingface_hub import notebook_login


# notebook_login()

In [ ]:

# 2. 필요한 라이브러리 import
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 3. 모델 및 토크나이저 불러오기
# Hugging Face에 공개된 LG EXAONE 모델 ID
model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

print(f"'{model_id}' 모델 로드를 시작합니다. 몇 분 정도 소요될 수 있습니다...")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드
# device_map="auto": 사용 가능한 GPU를 자동으로 할당
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

print("모델 로드가 완료되었습니다.")

'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct' 모델 로드를 시작합니다. 몇 분 정도 소요될 수 있습니다...


vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

model-00002-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

모델 로드가 완료되었습니다.


### 영상에서 정보 수집

In [ ]:
import os
import yt_dlp
import glob
from pydub import AudioSegment
import speech_recognition as sr
import wave, contextlib, webrtcvad
import json

YOUTUBE_URL = "https://www.youtube.com/watch?v=wv5fWjD-mLI"


# --- 2. 오디오 다운로드 ---
def download_audio_from_youtube(url, out_template='downloaded_audio.%(ext)s'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': out_template,
        'quiet': False,
        'no_warnings': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        candidates = glob.glob('downloaded_audio.*')
        if candidates:
            candidates.sort(key=os.path.getmtime, reverse=True)
            return candidates[0]
        return None
    except Exception as e:
        print("다운로드 에러:", e)
        return None


# --- 3. WAV 변환 ---
def convert_to_wav(input_filepath, wav_path='converted_audio.wav'):
    try:
        audio = AudioSegment.from_file(input_filepath)
        audio = audio.set_channels(1).set_frame_rate(16000)
        audio.export(wav_path, format='wav', parameters=["-acodec", "pcm_s16le"])
        return wav_path
    except Exception as e:
        print("변환 에러:", e)
        return None


# --- 4. VAD 기반 발화 단위 분리 ---
def read_wave(path):
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def vad_split(audio_path, aggressiveness=2):
    vad = webrtcvad.Vad(aggressiveness)
    audio, sample_rate = read_wave(audio_path)
    frame_duration = 30  # ms
    frame_size = int(sample_rate * frame_duration / 1000) * 2
    segments = []
    voiced_frames = []
    idx = 0
    start_time = None

    for i in range(0, len(audio), frame_size):
        frame = audio[i:i+frame_size]
        if len(frame) < frame_size:
            break
        is_speech = vad.is_speech(frame, sample_rate)
        if is_speech:
            if start_time is None:
                start_time = (i / frame_size) * (frame_duration / 1000.0)
            voiced_frames.append(frame)
        else:
            if voiced_frames:
                segment = b''.join(voiced_frames)
                out_path = f"vad_segment_{idx}.wav"
                with contextlib.closing(wave.open(out_path, 'wb')) as wf:
                    wf.setnchannels(1)
                    wf.setsampwidth(2)
                    wf.setframerate(sample_rate)
                    wf.writeframes(segment)

                end_time = (i / frame_size) * (frame_duration / 1000.0)
                segments.append((out_path, start_time, end_time))

                voiced_frames = []
                start_time = None
                idx += 1
    return segments


# --- 5. VAD 세그먼트 STT ---
def transcribe_vad_segments(wav_path, language='ko-KR'):
    segment_infos = vad_split(wav_path, aggressiveness=2)
    print("발화 단위 segment 개수:", len(segment_infos))

    r = sr.Recognizer()
    transcripts = []
    for seg_file, start, end in segment_infos:
        with sr.AudioFile(seg_file) as source:
            audio_data = r.record(source)
            try:
                text = r.recognize_google(audio_data, language=language)
            except sr.UnknownValueError:
                text = ""
            except sr.RequestError as e:
                text = f"[API 요청 실패: {e}]"

        transcripts.append({
            "file": seg_file,
            "text": text,
            "start": round(start, 2),
            "end": round(end, 2)
        })
    return transcripts


# --- 6. 실행 ---
if __name__ == "__main__":
    print("유튜브 오디오 다운로드...")
    downloaded = download_audio_from_youtube(YOUTUBE_URL)
    if not downloaded:
        raise RuntimeError("다운로드 실패")
    print("다운로드 완료:", downloaded)

    print("WAV 변환...")
    wav = convert_to_wav(downloaded)
    if not wav:
        raise RuntimeError("WAV 변환 실패")
    print("WAV 변환 완료:", wav)

    print("발화 단위 인식 시작...")
    transcripts = transcribe_vad_segments(wav, language="ko-KR")

    with open("vad_transcript.json", "w", encoding="utf-8") as f:
        json.dump(transcripts, f, ensure_ascii=False, indent=2)

    print("완료: vad_transcript.json 생성됨")


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


유튜브 오디오 다운로드...
[youtube] Extracting URL: https://www.youtube.com/watch?v=wv5fWjD-mLI
[youtube] wv5fWjD-mLI: Downloading webpage
[youtube] wv5fWjD-mLI: Downloading tv client config
[youtube] wv5fWjD-mLI: Downloading tv player API JSON
[youtube] wv5fWjD-mLI: Downloading web safari player API JSON
[youtube] wv5fWjD-mLI: Downloading player 0004de42-main
[youtube] wv5fWjD-mLI: Downloading m3u8 information
[info] wv5fWjD-mLI: Downloading 1 format(s): 96
[download] Sleeping 4.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 37
[download] Destination: downloaded_audio.mp4
[download] 100% of   30.81MiB in 00:00:02 at 11.11MiB/s                
[FixupM3u8] Fixing MPEG-TS in MP4 container of "downloaded_audio.mp4"
다운로드 완료: downloaded_audio.mp4
WAV 변환...
WAV 변환 완료: converted_audio.wav
발화 단위 인식 시작...
발화 단위 segment 개수: 22
완료: vad_transcript.json 생성됨


In [ ]:
import yt_dlp

YOUTUBE_URL = "https://www.youtube.com/watch?v=aG8ZUcikQcE"

def download_video(url, out_template="downloaded_video.%(ext)s"):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',  # 영상+음성 합쳐서 MP4
        'merge_output_format': 'mp4',
        'outtmpl': out_template,
        'quiet': False,
        'no_warnings': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        return ydl.prepare_filename(info)

if __name__ == "__main__":
    video_path = download_video(YOUTUBE_URL)
    print(f"✅ 다운로드 완료: {video_path}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=aG8ZUcikQcE
[youtube] aG8ZUcikQcE: Downloading webpage
[youtube] aG8ZUcikQcE: Downloading tv client config
[youtube] aG8ZUcikQcE: Downloading tv player API JSON
[youtube] aG8ZUcikQcE: Downloading web safari player API JSON
[youtube] aG8ZUcikQcE: Downloading m3u8 information
[info] aG8ZUcikQcE: Downloading 1 format(s): 399+251
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: downloaded_video.f399.mp4
[download] 100% of   13.22MiB in 00:00:00 at 38.58MiB/s  
[download] Destination: downloaded_video.f251.webm
[download] 100% of    4.30MiB in 00:00:00 at 61.99MiB/s  
[Merger] Merging formats into "downloaded_video.mp4"
Deleting original file downloaded_video.f251.webm (pass -k to keep)
Deleting original file downloaded_video.f399.mp4 (pass -k to keep)
✅ 다운로드 완료: downloaded_video.mp4


In [ ]:
!ffmpeg -i downloaded_video.mp4 -vf "fps=25" -c:v libx264 -preset fast -crf 23 -c:a copy fixed_video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from deepface import DeepFace
import cv2
import numpy as np

def analyze_frames_deepface(video_path, start_sec, end_sec, num_frames_to_sample=5, save_debug=True):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"⚠️ 비디오 열기 실패: {video_path}")
        return {"emotion_scores": {"neutral": 1.0}, "dominant_emotion": "neutral"}

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 25  # fallback

    start_frame = int(start_sec * fps)
    end_frame = int(end_sec * fps)
    frame_indices = np.linspace(start_frame, end_frame, num_frames_to_sample, dtype=int)

    all_scores = []
    os.makedirs("debug_frames", exist_ok=True)

    for idx, f_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f_idx)
        ret, frame = cap.read()
        if not ret or frame is None:
            print(f"⚠️ 프레임 추출 실패 (frame={f_idx})")
            continue

        # 저장
        if save_debug:
            fname = f"debug_frames/frame_{round(start_sec,2)}_{round(end_sec,2)}_{idx}.jpg"
            cv2.imwrite(fname, frame)

        try:
            analysis = DeepFace.analyze(
                frame,
                actions=["emotion"],
                detector_backend="opencv",
                enforce_detection=False
            )
            emo = analysis[0]["emotion"]
            all_scores.append(emo)
        except Exception as e:
            print(f"⚠️ DeepFace 분석 실패 (frame={f_idx}): {e}")

    cap.release()

    if all_scores:
        keys = all_scores[0].keys()
        final_scores = {k: float(np.mean([emo[k] for emo in all_scores])) for k in keys}
        dominant = max(final_scores, key=final_scores.get)
    else:
        final_scores = {"neutral": 1.0}
        dominant = "neutral"

    return {"emotion_scores": final_scores, "dominant_emotion": dominant}



def map_emotions_to_transcript(transcript_json, video_path):
    with open(transcript_json, "r", encoding="utf-8") as f:
        transcripts = json.load(f)

    results = []

    for seg in transcripts:
        text = seg.get("text", "").strip()
        if not text:
            continue  # 빈 텍스트 스킵

        start_sec = float(seg.get("start", 0.0))
        end_sec = float(seg.get("end", start_sec + 0.5))

        emo_scores = analyze_frames_deepface(
            video_path,
            start_sec=start_sec,
            end_sec=end_sec,
            num_frames_to_sample=5,
            save_debug=True  # 프레임 저장 활성화
        )

        results.append({
            "file": seg.get("file"),
            "text": text,
            "start": round(start_sec, 2),
            "end": round(end_sec, 2),
            "emotion_scores": emo_scores["emotion_scores"],
            "dominant_emotion": emo_scores["dominant_emotion"]
        })

    return results


if __name__ == "__main__":
    transcript_path = "/content/vad_transcript.json"
    video_path = "/content/fixed_video.mp4"

    mapped_results = map_emotions_to_transcript(transcript_path, video_path)

    with open("mapped_transcript.json", "w", encoding="utf-8") as f:
        json.dump(mapped_results, f, ensure_ascii=False, indent=2)

    print("✅ 완료: mapped_transcript.json 생성됨")
    print("📸 디버그 프레임은 debug_frames/ 폴더에 저장됨")


25-09-28 01:34:06 - Directory /root/.deepface has been created
25-09-28 01:34:06 - Directory /root/.deepface/weights has been created
25-09-28 01:34:09 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 403MB/s]


✅ 완료: mapped_transcript.json 생성됨
📸 디버그 프레임은 debug_frames/ 폴더에 저장됨


### 프롬포트 엔지니어링 가능한 코드

In [ ]:
import torch
import json
import re
from transformers import AutoModelForCausalLM, AutoTokenizer


def create_prompt_for_llm(text: str):
    """
    입력: 한국어 문장 (STT 결과)
    출력: JSON 한 줄 (ksl_gloss 변환 결과)
    """
    prompt = f"""
문장을 한국수어 글로스(KSL Gloss)로 변환하세요.
# 변환 규칙
1.  **핵심 내용 추출**: 문장의 핵심 의미를 전달하는 명사, 동사, 형용사 위주로 변환합니다.
2.  **어순 재배열**: 문장의 주제(Topic)나 시간, 장소 정보를 가장 앞에 배치하고, 주어-목적어-서술어(SOV) 어순을 따릅니다.
3.  **비수지 신호(NMS) 태그**: 문장 끝에 종류에 따라 (평서), (의문), (부정) 태그를 반드시 붙입니다.
    - (의문): '왜', '언제' 등이 없어도 질문의 뉘앙스이면 사용합니다.
    - (부정): '못하다', '아니다' 등 부정의 의미가 있으면 사용합니다.
4.  **인물/사물 지정(INDEX)**: 처음 등장하는 인물/사물은 '이름(INDEX-1)'과 같이 지정할 수 있습니다.
5.  **출력 형식**: 다른 설명 없이 JSON 한 줄만 출력합니다.


입력: "{text}"

출력:
{{"ksl_gloss": "..."}}
"""
    return prompt.strip()


def load_llm(model_id):
    print(f"'{model_id}' 모델 로드를 시작합니다...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto"
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("모델 로드가 완료되었습니다.")
    return model, tokenizer


def parse_llm_output(raw_output: str):
    """LLM 출력에서 JSON만 추출"""
    try:
        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        return None
    except Exception:
        return None


def fallback_gloss(text: str) -> str:
    """
    LLM 실패 시 기본 전처리 방식으로 변환
    """
    text = re.sub(r"(은|는|이|가|을|를|과|와|에|에서|으로|에게|도|만|까지|부터)", "", text)  # 조사 제거
    text = re.sub(r"[^가-힣\s]", "", text)  # 특수문자 제거
    words = text.strip().split()
    return "/".join(words)


def run_llm_on_segments(model, tokenizer, segments, max_new_tokens=64):
    results = []
    for i, seg in enumerate(segments):
        stt_text = seg.get("text", "")
        prompt = create_prompt_for_llm(stt_text)

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
        raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        parsed = parse_llm_output(raw_output)
        if parsed is None or "ksl_gloss" not in parsed:
            gloss = fallback_gloss(stt_text)
        else:
            gloss = parsed.get("ksl_gloss", "")

        # ✅ dominant_emotion을 최종 결과에 포함
        result = {
            "text": stt_text,
            "ksl_gloss": gloss,
            "dominant_emotion": seg.get("dominant_emotion", "neutral")  # 기본값 neutral
        }

        # (선택) 원하면 file/start/end 같은 원본 메타데이터도 함께 보존 가능
        for k in ("file", "start", "end", "emotion_scores"):
            if k in seg:
                result[k] = seg[k]

        results.append(result)

        print(f"[{i+1}/{len(segments)}] {result['text']} → {result['ksl_gloss']} | emotion={result['dominant_emotion']}")

    return results


if __name__ == "__main__":
    transcript_path = "/content/mapped_transcript.json"
    model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

    # 1. 모델 로드
    model, tokenizer = load_llm(model_id)

    # 2. transcript 불러오기 (여기엔 dominant_emotion이 포함되어 있어야 함)
    with open(transcript_path, "r", encoding="utf-8") as f:
        segments = json.load(f)

    # 3. LLM 실행 (+ dominant_emotion 포함된 결과 생성)
    gloss_results = run_llm_on_segments(model, tokenizer, segments)

    # 4. 결과 저장
    with open("ksl_gloss_results.json", "w", encoding="utf-8") as f:
        json.dump(gloss_results, f, ensure_ascii=False, indent=2)

    print("✅ 완료: ksl_gloss_results.json 생성됨 (텍스트 + 글로스 + dominant_emotion 포함)")


'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct' 모델 로드를 시작합니다...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

모델 로드가 완료되었습니다.
[1/5] 미안해 많이 기다렸어 왜 늦었어 → 미안해/많/기다렸어/왜/늦었어 | emotion=sad
[2/5] 내가 아저씨라 그런가 그럴 수도 있어 모가지가 아프네 병원 안 갔어 응 저번 주 평일에 시간 없었어 그럼 오늘도 아프면 집에서 쉬지 그냥 사 오겠다 그 정도는 아닌 거 같아 그래 병원가 봐 내 커피 시키고 올게 응 싸우는 거야 싸우는 거야 아 여보세요 어 진아 어 우리 집 파마 했다고 사진 보내 봐 돈 얼마 들었어 너 머리 길어서 돈 많이 나겠다 알겠어 끊어 진아 전화 왔어 그래 사진 보내 보라 그러지 그랬는데 나중에 다시 전화한다고 끊었어 바쁜가 보지 그런가 봐 오빠 나 살쪘어 왜 아니 나 사진 찍은 거 보는데 왜 이렇게 살이 쪘지 그런가 지금 이게 짱 걸 먹어서 부은 건지 아니면 진짜 살이 찐 건지 잘 모르겠어 짱구랑 먹지는 않았어 그래 손 줘 봐 아니 2kg 빠졌어 근데 내 눈치 봐서 그렇게 말한 거지 아니 귀여워 우리 오늘 뭐 하지 → ... | emotion=happy
[3/5] 회사 회사 왜 저번 주에 지각했는데 이번 주에 내지 말라고 근데 저번 주에 자기가 좀 늦게 자긴 했어 왜냐면 그 사람들도 똑같이 일하고 출근하는데 나보다 후임이 맨날 나보다 늦게 오면 나 같아도 화날 것 같긴 해 아이고 진짜 싸 → 회사/회사/왜/저번/주/지각했데/번/주/내지/말라고/근데/저번/주/자기/좀/늦게/자긴/했어/왜냐면/그/사람들/똑같/일하고/출근하데/나보다/후임/맨날/나보다/늦게/오면/나/같아/화날/것/같긴/해/아고/진짜/싸 | emotion=sad
[4/5] 병원 예약해 놨어 자기 평일에 또 시간 없다 할 것 같아 → 병원/예약해/놨어/자기/평일/또/시간/없다/할/것/같아 | emotion=neutral
[5/5] 무거워 하자 → 무거워/하자 | emotion=neutral
✅ 완료: ksl_gloss_results.json 생성됨 (텍스트 + 글로스 + dominant_emotion 포함)


## 파인튜닝 실험

## 글로스 -> 몸 좌표

## 글로스 -> vrm 출력